### Import necessary packages

In [12]:
from pyspark import *
from pyspark.sql import *

### Initializing spark session

In [13]:
spark = SparkSession.builder \
        .appName("clean to silver stage") \
        .master("local[3]") \
        .getOrCreate()

### Reading the cleaned data

In [14]:
country_companies_silver = spark.read \
                            .format("parquet") \
                            .load("cleaned_data/")

### Creating the temp view 

In [15]:
country_companies_silver.createOrReplaceTempView("country_companies_data")

### Using spark sql to find the total profit for each country

In [33]:
country_total_profit = spark.sql("""
            select country, round(sum(profits_in_billions),2) as total_profit_in_billions 
            from country_companies_data 
            group by country;""")


### Writing the country profit data

In [39]:
country_total_profit.write \
        .format("parquet") \
        .mode("overwrite") \
        .option("path" ,"country_profit/") \
        .save()

### Using spark sql to find the total profit for each continent

In [41]:
continent_total_profit = spark.sql("""
                        select continent , round(sum(profits_in_billions),2) as total_profit_in_billions 
                        from country_companies_data
                        group by continent ;
                        """)

### Writing the continent profit data

In [42]:
continent_total_profit.write \
        .format("parquet") \
        .mode("overwrite") \
        .option("path" ,"continent_profit/") \
        .save()